In [1]:
import numpy as np
import json
from random import randint, shuffle

### GAME CLASS

In [2]:
class Game:
    mask = None
    
    @classmethod
    def getField(cls, v = 0):
        f = np.full((7, 7), v, dtype=int)    
        for i in [0, 1, 5, 6]:
            for j in [0, 1, 5, 6]:
                f[i, j] = -1
        
        if cls.mask is None:
            cls.mask = (f != -1).flatten()
        
        return f  
    
    def __init__(self):
        self.field = self.getField(1)
        self.field[3, 3] = 0
        self.turns = []
        
    def __str__(self):
        s = ''
        lastTurn = None
        if len(self.turns) > 0:
            lastTurn = self.turns[-1]
        for i in range(7):
            for j in range(7):
                v = self.field[i, j]
                if v == -1:
                    c = ' '
                elif v == 1:
                    c = '◉'
                elif v == 0:
                    c = '◌'
                if lastTurn and (i, j) in lastTurn:
                    c = "\x1b[31m%s\x1b[0m" % c
                s += '%s ' % c 
            s += '\n'
        return s
    
    def printTurns(self):
        gt = Game() 
        for e in self.turns:
            gt.turn(e[0], e[1])
            print(gt)

    def asBoolArray(self):
        return self.field.flatten()[Game.mask].astype(bool)
    
    # def asTuple(self):
    #     return tuple(self.asBoolArray())
    
    def asInt(self):
        return sum([b << i for i, b in enumerate(self.asBoolArray())])
    
    def isFinished(self, strict = False):
        if sum(self.asBoolArray()) > 1:
            return False
        
        if not strict:
            return True
        
        if self.field[3, 3] == 1:
            return True
        
        return False

    def isFinished2(self, field):
        return np.array_equal(self.field, field)
    
    def getFieldsWithValue(self, v):
        return list(zip(*np.where(self.field == v)))
    
    def turn(self, f, t):
        (fx, fy), (tx, ty) = (f, t)
        self.field[fx, fy] = 0
        self.field[(fx + tx)//2, (fy + ty)//2] = 0
        self.field[tx, ty] = 1
        self.turns.append((f, t))
    
    def undoLastTurn(self):
        (fx, fy), (tx, ty) = self.turns.pop()
        self.field[fx, fy] = 1
        self.field[(fx + tx)//2, (fy + ty)//2] = 1
        self.field[tx, ty] = 0
    
    def getPossibleTurns(self):
        td = {}
        ts = []
        l0 = self.getFieldsWithValue(0)
        l1 = self.getFieldsWithValue(1)
        # [((2, 0), (1, 0)), ((0, 2), (0, 1))
        if len(l0) < len(l1):
            for x, y in l0:
                t = []
                if (x + 2, y) in l1 and (x + 1, y) in l1:
                    t.append((x + 2, y))
                if (x - 2, y) in l1 and (x - 1, y) in l1:
                    t.append((x - 2, y))
                if (x, y + 2) in l1 and (x, y + 1) in l1:
                    t.append((x, y + 2))
                if (x, y - 2) in l1 and (x, y - 1) in l1:
                    t.append((x, y - 2))
                #td[(x,y)] = t
                ts.extend(list(zip(t, [(x, y)]*len(t))))
        else:
            for x, y in l1:
                t = []
                if (x + 2, y) in l0 and (x + 1, y) in l1:
                    t.append((x + 2, y))
                if (x - 2, y) in l0 and (x - 1, y) in l1:
                    t.append((x - 2, y))
                if (x, y + 2) in l0 and (x, y + 1) in l1:
                    t.append((x, y + 2))
                if (x, y - 2) in l0 and (x, y - 1) in l1:
                    t.append((x, y - 2))
                #td[(x,y)] = t
                ts.extend(list(zip([(x, y)]*len(t), t)))
        return ts
            

### TEST GAME CLASS

In [3]:
g = Game()
print(g)
ts = g.getPossibleTurns()
t = ts[0]
print(t)
g.turn(*t)
print(g)
g.asInt()
g.undoLastTurn()
print(g)
g.turns

    ◉ ◉ ◉     
    ◉ ◉ ◉     
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
◉ ◉ ◉ ◌ ◉ ◉ ◉ 
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
    ◉ ◉ ◉     
    ◉ ◉ ◉     

((5, 3), (3, 3))
    ◉ ◉ ◉     
    ◉ ◉ ◉     
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
◉ ◉ ◉ ◌ ◉ ◉ ◉ 
    ◉ ◌ ◉     
    ◉ ◉ ◉     

    ◉ ◉ ◉     
    ◉ ◉ ◉     
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
◉ ◉ ◉ ◌ ◉ ◉ ◉ 
◉ ◉ ◉ ◉ ◉ ◉ ◉ 
    ◉ ◉ ◉     
    ◉ ◉ ◉     



[]

In [ ]:
targetField = Game()
targetField.field = Game.getField(0)
# targetField.field[1, 3] = 0
# targetField.field[2, 2] = 0
# targetField.field[2, 3] = 0
# targetField.field[2, 4] = 0
# targetField.field[3, 2] = 0
targetField.field[3, 3] = 1
# targetField.field[3, 4] = 0
# targetField.field[4, 2] = 0
# targetField.field[4, 3] = 0
# targetField.field[4, 4] = 0
# targetField.field[5, 3] = 0

# targetField.field[3, 5] = 0
# targetField.field[3, 1] = 0
print(targetField)
sum(targetField.asBoolArray())
targetField.sum = 33 - sum(targetField.asBoolArray())
targetField.sum

### BRUTE FORCE WITH MEMORY

In [ ]:
game = Game()
mem = set()
def rec(game, i = 1):
    if i == targetField.sum:
        return game.isFinished2(targetField.field)
    
    for turn in game.getPossibleTurns():
        game.turn(*turn)
        fieldInt = game.asInt()
        if fieldInt not in mem:
            mem.add(fieldInt)
            if rec(game, i + 1):
                return True
        game.undoLastTurn()
        
    return False
        
%prun print(rec(game))
#print(len(mem))


### BRUTE FORCE

In [ ]:
g = Game()
#g.field = np.zeros((7, 7), dtype=int)
#g.field[4, 4] = 1
#g.field[4, 5] = 1
#g.field[4, 3] = 1

im = 100
s = False
mem = set()
def rec(g, i = 0):
    ts = g.getPossibleTurns()
    shuffle(ts)
    
#     global im, s
#     if i < im and s:
#         print(i)
#         im = i
        
    if len(ts) == 0:
        # s = True
        return g.isFinished()
    
    for t in ts:
        g.turn(*t)
        tf = g.asInt()
        if tf not in mem:
            mem.add(tf)
            a = rec(g, i + 1)
            if a:
                return True
        g.undoLastTurn()
        
    return False
        
print(rec(g))
print(len(mem))

In [ ]:
print(g)

In [ ]:
g.printTurns()

In [ ]:
# RANDOM
while True:
    g = Game()
    ats = []
    while True:
        ts = g.getPossibleTurns()
        if len(ts) == 0:
            break
        t = ts[randint(0, len(ts)-1)]
        ats.append(t)
        g.turn(*t)
    l = len(g.getFieldsWithValue(1))
    if l <= 2:
        #break
        print(l)
    if l == 1:
        break
print(g)